In [1]:
## SIMULATION PARAMETERS
verbosity         = 8        # Set debug print statement verbosity level (0 = Standard, -1 = Off)
use_mass_units    = True     # Toggle whether calculations / results are given in units of pi-axion mass (True) or eV (False)
use_natural_units = True     # Toggle whether calculations / results are given in c = h = G = 1 (True) or SI units (False)   || NOTE: full SI/phsyical unit support is still WIP!!
save_output_files = True     # Toggle whether or not the results from this notebook run are written to a data directory
save_plots        = True     # Toggle whether or not to generate and save plots
use_logsumexp     = True     # Toggle whether or not numerical integrations are performed in log-scale number space (increases computation time)

## CONFIG PARAMETERS
config_name = "piaxiverse_FRM"   # Descriptive name for the given parameter case. Output files will be saved in a directory with this name.
seed = None
#seed = 269934808235809364794393739371438879116                      # rng_seed, integer value (None for random)
#seed = 191281265662032962853842089260008419904
num_cores = 100              # Number of parallel threads available
method = "BDF"               # numerical integration method, to be passed to scipy.solve_ivp()

## Scenario to run
# NOTE: (NAME) indicates NAME is an alias of the previously listed item.
scenario = "SU6"           # one of: SINGLE, QCD, (AXION), SIMPLE, FULL, SU3, SU6, (SAMPLED), REALS, NEUTRALS, COMPLEX, CHARGED

In [2]:
#%cd ~/projects/pi-axiverse/

In [3]:
#%load piaxiverse.py

In [4]:
#%module load miniconda3/23.11.0s
#%module load texlive
#%source /oscar/runtime/software/external/miniconda3/23.11.0/etc/profile.d/conda.sh
#%conda activate piaxiverse

In [5]:
# Optionally pregenerate a list of function calls for parameter space sampling
import numpy as np

# True  = call pyAxiverse main loop separately for each combination of parameters
# False = iterate through parameters within pyAxiverse main loop itself (WARNING: Potential OoM errors)
pregen_commands = True

# (Log-scale) min and max ranges over which to sample
density_range = [0, 40]        # Density [log GeV/cm^3]
density_num   = 10
qmass_range   = [-10, -80]     # Dark quark mass scale [log eV]
#qmass_range   = [-37, -37]     # Dark quark mass scale [log eV]
qmass_num     = 5
#qmass_num     = 1
#epsilon_range = [0, -30]       # Millicharge [log]
epsilon_range = [0, 0]       # Millicharge [log]
#epsilon_num   = 5
epsilon_num   = 1
Fpi_range     = [0, 30]       # Pi-Axion decay constant [log GeV]
Fpi_num       = 10
#L3_range      = [0, 60]        # Pi-Axion charged effective coupling [log GeV]
#L3_num        = 3
L3_range      = [0, 0]        # Pi-Axion charged effective coupling [log GeV]
L3_num        = 1
#L4_range      = [0, 60]        # Pi-Axion neutral effective coupling [log GeV]
#L4_num        = 5
L4_range      = [10, 10]        # Pi-Axion neutral effective coupling [log GeV]
L4_num        = 1

# Simulation bounds and granularity
num_times  = 1000   # Number of timesteps
max_tval   = 100    # Max time value (in time units)
num_kmodes = 200    # Number of k_modes
kmode_res  = 0.1    # k_mode step-size

In [6]:
# Toggle whether calculations are performed in natural units (h=c=1) or physical units
# NOTE: Currently, natural units also requires unitless masses (and vice versa)
if use_mass_units or use_natural_units:
    PIAXI_UNITS = "--use_mass_units --use_natural_units"
else:
    PIAXI_UNITS = "--no-use_mass_units --no-use_natural_units"

# RNG seed
if seed is None:
    PIAXI_SEED = ""
else:
    PIAXI_SEED = f"--seed {seed} "

# Whether or not to save output files and plots
if save_plots:
    PIAXI_PLOTS = "--make_plots --no-show_plots"
else:
    PIAXI_PLOTS = "--no-make_plots --no-show_plots"
if save_output_files:
    PIAXI_SAVE  = f"--save_output_files {PIAXI_PLOTS}"
else:
    PIAXI_SAVE  = "--no-save_output_files --no-make_plots --no-show_plots"

if use_logsumexp:
    PIAXI_LOGSCALE="--use_logsumexp "
else:
    PIAXI_LOGSCALE=""

# Determine number of surviving pi-axion species and parameter distributions
PIAXI_JOB_SUFFIX = ""
PIAXI_FIT_F = ""
PIAXI_MASKS = ""
scenario_in = scenario.upper()
if scenario_in in ["EMPTY", "VACUUM"]: # (NOTE: May raise errors, not fully implemented)
    PIAXI_DQMC = "0 0 0 0 0 0"
    PIAXI_JOB_SUFFIX = "_vacuum"
elif scenario_in in ["0", None, "NONE", "NULL", "DEFAULT", ""]:
    PIAXI_DQMC = "1 1 1 0 0 0"
elif scenario_in in ["SINGLE"]:
    PIAXI_DQMC = "x x 0 0 0 0"
    PIAXI_JOB_SUFFIX = "_single"
elif scenario_in in ["QCD", "AXION", "QCD_AXION", "KSVZ"]:
    PIAXI_DQMC = "0.5 0.5 0 0 0 0"
    PIAXI_JOB_SUFFIX = "_qcd"
    PIAXI_FIT_F = "--fit_QCD"
    PIAXI_MASKS = "--mask_complex --mask_charged"
elif scenario_in in ["SIMPLE"]:
    PIAXI_DQMC = "1 1 1 0 0 0"
    PIAXI_JOB_SUFFIX = "_simple"
elif scenario_in in ["FULL", "DEGENERATE"]:
    PIAXI_DQMC = "1 1 1 1 1 1"
    PIAXI_JOB_SUFFIX = "_fully_degenerate"
elif scenario_in in ["SU3"]:
    PIAXI_DQMC = "x x x 0 0 0"
    PIAXI_JOB_SUFFIX = "_SU3"
elif scenario_in in ["SAMPLED", "SU6"]:
    PIAXI_DQMC = "x x x x x x"
    PIAXI_JOB_SUFFIX = "_SU6"
elif scenario_in in ["REALS", "AXIVERSE"]:
    PIAXI_DQMC = "x x x x x x"
    PIAXI_JOB_SUFFIX = "_reals"
    PIAXI_MASKS = "--mask_complex --mask_charged"
elif scenario_in in ["NEUTRALS", "ALL_NEUTRALS"]:
    PIAXI_DQMC = "x x x x x x"
    PIAXI_JOB_SUFFIX = "_neutrals"
    PIAXI_MASKS = "--mask_charged"
elif scenario_in in ["COMPLEX"]:
    PIAXI_DQMC = "x x x x x x"
    PIAXI_JOB_SUFFIX = "_complex"
    PIAXI_MASKS = "--mask_reals"
elif scenario_in in ["COMPLEX_NEUTRALS"]:
    PIAXI_DQMC = "x x x x x x"
    PIAXI_JOB_SUFFIX = "_complex_neutrals"
    PIAXI_MASKS = "--mask_reals --mask_charged"
elif scenario_in in ["CHARGED"]:
    PIAXI_DQMC = "x x x x x x"
    PIAXI_JOB_SUFFIX = "_charged"
    PIAXI_MASKS = "--mask_reals --mask_complex"

PIAXI_RES_ARGS = f"--t {max_tval} --tN {num_times} --kN {num_kmodes} --k_res {kmode_res}"
PIAXI_SYS_NAME = f"{config_name}{PIAXI_JOB_SUFFIX}"

In [7]:
import os
param_list  = lambda x_range, x_num: np.linspace(x_range[0], x_range[1], num=x_num) if x_num > 1 else np.array([x_range[0]], dtype=np.float64)

num_samples = 3   # how many times to rerun a given input configuration, resampling any random variables

# Gather relevant args to pass to command line
PIAXI_INPUT_ARGS=f"--int_method {method} {PIAXI_SEED}{PIAXI_LOGSCALE}{PIAXI_UNITS} --verbosity {verbosity} {PIAXI_RES_ARGS} --config_name {PIAXI_SYS_NAME} {PIAXI_SAVE}"
#PIAXI_SLURM_ARGS=f"--num_cores {PIAXI_N_CORES} --num_nodes {PIAXI_N_NODES} --job_qos {PIAXI_JOB_QOS} --mem_per_core {PIAXI_COREMEM}"

if pregen_commands:
    density_list = 10**param_list(density_range, density_num)
    qmass_list   = 10**param_list(qmass_range, qmass_num)
    epsilon_list = 10**param_list(epsilon_range, epsilon_num)
    Fpi_list     = 10**param_list(Fpi_range, Fpi_num)
    L4_list      = 10**param_list(L4_range, L4_num)

    # No need to sample L3 if there are never any charged species permitted in the subtheory
    if PIAXI_JOB_SUFFIX in ["_complex_neutrals", "_neutrals", "_reals", "_qcd"]:
        L3_list = [1.]
    else:
        L3_list = 10**param_list(L3_range, L3_num)

    if verbosity >= 5:
        print('Ranges')
        print('--------------------------------------------------------------------')
        print('density: ', density_list)
        print('qmass: ', qmass_list)
        print('epsilon: ', epsilon_list)
        print('Fpi: ', Fpi_list)
        print('L3:  ', L3_list)
        print('L4:  ', L4_list)
        print('--------------------------------------------------------------------')
    tot_count = np.prod([len(x) for x in [density_list, qmass_list, epsilon_list, Fpi_list, L3_list, L4_list]]) * num_samples
    print('Total number of function calls required:    N = %d' % tot_count)

argset = [(rho, qm, eps, Fpi, L3, L4) for rho in density_list for qm in qmass_list for eps in epsilon_list for Fpi in Fpi_list for L3 in L3_list for L4 in L4_list for n in range(num_samples)]

argfile_dir  = '~/scratch'
argfile_path = os.path.join(os.path.expanduser('~'), '/'.join(argfile_dir.split('/')[1:]), 'ARGFILES')
if not(os.path.exists(argfile_path)):
    os.makedirs(argfile_path)
argfile_name = os.path.join(argfile_path, PIAXI_SYS_NAME)

with open(argfile_name, 'w') as f:
    for rho_in, qm_in, eps_in, Fpi_in, L3_in, L4_in in argset:
        # Gather parameter space args to pass to command line
        PIAXI_DENSITY_ARGS = f"--rho {rho_in}"           # Density [GeV] || For QCD axion case ~ (amp_a)^2*m_a / 2
        PIAXI_F_ARGS = f"--F {Fpi_in} {PIAXI_FIT_F}"     # F_pi [GeV]    || For QCD axion case ~ 2/g_a
        PIAXI_M_ARGS = f"--m_scale {qm_in}"              # m_I [eV]      || For QCD axion case ~ (m_a)^2 / F_pi
        PIAXI_L_ARGS = f"--L3 {L3_in} --L4 {L4_in}"      # Pi-Axiverse coupling constants: Lambda_3 and Lambda_4 [GeV]
        PIAXI_EPS_ARGS = f"--eps {eps_in}"               # Millicharge (unitless)

        PIAXI_PARAM_ARGS=f"{PIAXI_DENSITY_ARGS} {PIAXI_F_ARGS} {PIAXI_M_ARGS} {PIAXI_L_ARGS} {PIAXI_EPS_ARGS} --dqm_c {PIAXI_DQMC} {PIAXI_MASKS}"

        f.write(f"python piaxiverse.py {PIAXI_INPUT_ARGS} {PIAXI_PARAM_ARGS} --no-skip_existing \n")

print('To sample this pi-axiverse parameter space, execute the following command: ')
print('\n')
scriptdir = os.path.join(os.path.expanduser('.'), '/'.join(['scripts','sample_pi_axiverse.sh']))
#print(f'      sbatch --array=1-{tot_count} --job-name={config_name}{PIAXI_JOB_SUFFIX}_array {scriptdir} {argfile_name}')
print(f'      sbatch --job-name={config_name}{PIAXI_JOB_SUFFIX}_array {scriptdir} {argfile_name} 1 {tot_count}')
print('\n')
print('--------------------------------------------------------------------')


Ranges
--------------------------------------------------------------------
density:  [1.00000000e+00 2.78255940e+04 7.74263683e+08 2.15443469e+13
 5.99484250e+17 1.66810054e+22 4.64158883e+26 1.29154967e+31
 3.59381366e+35 1.00000000e+40]
qmass:  [1.00000000e-10 3.16227766e-28 1.00000000e-45 3.16227766e-63
 1.00000000e-80]
epsilon:  [1.]
Fpi:  [1.00000000e+00 2.15443469e+03 4.64158883e+06 1.00000000e+10
 2.15443469e+13 4.64158883e+16 1.00000000e+20 2.15443469e+23
 4.64158883e+26 1.00000000e+30]
L3:   [1.]
L4:   [1.e+10]
--------------------------------------------------------------------
Total number of function calls required:    N = 1500
To sample this pi-axiverse parameter space, execute the following command: 


      sbatch --job-name=piaxiverse_FRM_SU6_array ./scripts/sample_pi_axiverse.sh /users/sloane1/scratch/ARGFILES/piaxiverse_FRM_SU6 1 1500


--------------------------------------------------------------------
